In [20]:
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
from numba import jit
from tqdm import tqdm
from itertools import combinations, permutations
import time

In [21]:
data_A = np.loadtxt('../data/TSPA.csv', delimiter=';').astype(np.int64)
data_B = np.loadtxt('../data/TSPB.csv', delimiter=';').astype(np.int64)
data_C = np.loadtxt('../data/TSPC.csv', delimiter=';').astype(np.int64)
data_D = np.loadtxt('../data/TSPD.csv', delimiter=';').astype(np.int64)

In [22]:
def create_cost_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2) + cost.reshape(1, -1)).round().astype(np.int64)

def create_dist_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    #cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2)).round().astype(np.int64)

In [23]:
cost_matrix_A = create_cost_matrix(data_A)

In [24]:
cost_matrix_A

array([[  84, 2032, 2098, ..., 4159, 3783, 1514],
       [1633,  483, 2398, ..., 3349, 2266,  817],
       [ 720, 1419, 1462, ..., 3640, 3149,  964],
       ...,
       [2782, 2371, 3641, ..., 1461, 2908, 2554],
       [2558, 1440, 3302, ..., 3060, 1309, 1773],
       [1234,  936, 2062, ..., 3651, 2718,  364]], dtype=int64)

In [25]:
cost_matrix_B = create_cost_matrix(data_B)

In [26]:
cost_matrix_C = create_cost_matrix(data_C)

In [27]:
cost_matrix_D = create_cost_matrix(data_D)

In [28]:
dist_matrix_A = create_dist_matrix(data_A)
dist_matrix_B = create_dist_matrix(data_B)
dist_matrix_C = create_dist_matrix(data_C)
dist_matrix_D = create_dist_matrix(data_D)

In [29]:
def plot(data, solution):
    data_ordered = np.array([data[i] for i in solution])
    all_data = np.array([data[i] for i in range(200)])

    plt.figure(figsize=(10, 10), dpi=80)

    plt.scatter(data_ordered[:,0], data_ordered[:,1], s=data_ordered[:,2]/data_ordered[:,2].max()*200, c='b')
    plt.scatter(all_data[:,0], all_data[:,1], s=all_data[:,2]/all_data[:,2].max()*200, c='b')
    plt.plot(data_ordered[:,0], data_ordered[:,1], 'y-')
    plt.plot([data_ordered[0,0], data_ordered[-1,0]], [data_ordered[0,1], data_ordered[-1,1]], 'y-')
    plt.show()

In [30]:
def calculate_performance(cycle, cost_matrix):
    total_sum = 0
    for i in range(len(cycle)-1):
        total_sum += cost_matrix[cycle[i], cycle[i+1]]
    total_sum += cost_matrix[cycle[-1], cycle[0]]
    return total_sum

In [31]:
@jit()
def random_solution(cost_matrix, limit=100):
    random_solution_list = list(range(0,len(cost_matrix)))
    random.shuffle(random_solution_list)
    return np.array(random_solution_list)[:limit]

solution = random_solution(cost_matrix_A, 100)
print(solution)

[188  73  29  24  21 194 122  61 148 111 112  57  75 117 184 177 198  99
  71  95 147  85  44  72  17 143 195  38 186 157  91  59 189  87  74 161
   1 175   9   0  97  48  13  84 109 167 164  65 144  53 106  14  89  98
 197   7  55  62  63  15 182  28  33 142  30 166  41  77 129  81 118 113
  68  11 135 124  58  96  20 151 132 140 131  76   5  31  26  49 130   2
 103 190  19 181  27 119 125 136 183 115]


C:\Users\hubra\AppData\Local\Temp\ipykernel_8412\3945026401.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "random_solution" failed type inference due to: No implementation of function Function(<bound method Random.shuffle of <random.Random object at 0x000001A54B4F5400>>) found for signature:
 
 >>> shuffle(list(int64)<iv=None>)
 
There are 2 candidate implementations:
    - Of which 2 did not match due to:
    Overload in function 'shuffle_impl': File: numba\cpython\randomimpl.py: Line 1230.
      With argument(s): '(list(int64)<iv=None>)':
     Rejected as the implementation raised a specific error:
       TypeError: The argument to shuffle() should be a buffer type
  raised from C:\Users\hubra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cpython\randomimpl.py:1206

During: resolving callee type: Function(<bound method Random.shuffle of <rand

In [32]:
@jit()
def greedy_cycle(cost_matrix, current_id, limit=100):
    all_ids = set(list(range(0,len(cost_matrix))))
    all_ids.remove(current_id)
    solution = [current_id]
    
    for _ in range(1):
        min_val = 99999
        min_id = -1
        for next_id in all_ids:
            if cost_matrix[current_id][next_id] < min_val:
                min_val = cost_matrix[current_id][next_id]
                min_id = next_id
        solution.append(min_id)
        all_ids.remove(min_id)
        current_id = min_id
    
    while len(solution) < limit:
        min_delta = 99999
        min_id = -1
        insert_id = -1
        for i in range(len(solution)-1):
            for next_id in all_ids:
                delta = cost_matrix[solution[i]][next_id] + cost_matrix[next_id][solution[i+1]] - cost_matrix[solution[i]][solution[i+1]]
                if delta < min_delta:
                    min_delta = delta
                    min_id = next_id
                    insert_id = i
        for next_id in all_ids:
            delta = cost_matrix[solution[-1]][next_id] + cost_matrix[next_id][solution[0]] - cost_matrix[solution[-1]][solution[0]]
            if delta < min_delta:
                min_delta = delta
                min_id = next_id
                insert_id = i
        solution.insert(insert_id+1, min_id)
        all_ids.remove(min_id)

    return np.array(solution)

In [33]:
# swap node in cycle and unused node
def inter_exchange_delta(cycle, cost_matrix, used_cycle_id, unused_node):
    if used_cycle_id == len(cycle)-1:
        return cost_matrix[cycle[used_cycle_id-1], unused_node] + cost_matrix[unused_node, cycle[0]] - cost_matrix[cycle[used_cycle_id-1], cycle[used_cycle_id]] - cost_matrix[cycle[used_cycle_id], cycle[0]]
    return cost_matrix[cycle[used_cycle_id-1], unused_node] + cost_matrix[unused_node, cycle[used_cycle_id+1]] - cost_matrix[cycle[used_cycle_id-1], cycle[used_cycle_id]] - cost_matrix[cycle[used_cycle_id], cycle[used_cycle_id+1]]

In [34]:
def intra_two_nodes_exchange_delta(cycle, cost_matrix, node1, node2, pri = False):
    # print(node1, node2)
    if node1 == len(cycle)-1:
        node1plus = 0
    else:
        node1plus = node1 + 1
    
    if node2 == len(cycle)-1:
        node2plus = 0
    else:
        node2plus = node2 + 1

    if abs(node1-node2) == 1:
        if node1 > node2:
            node1, node2 = node2, node1
            node1plus, node2plus = node2plus, node1plus
        return cost_matrix[cycle[node1-1], cycle[node2]] + cost_matrix[cycle[node2], cycle[node1]] + cost_matrix[cycle[node1], cycle[node2plus]] - cost_matrix[cycle[node1-1], cycle[node1]] - cost_matrix[cycle[node1], cycle[node2]] - cost_matrix[cycle[node2], cycle[node2plus]]
    if abs(node1-node2) == len(cycle)-1:
        if node1 < node2:
            node1, node2 = node2, node1
            node1plus, node2plus = node2plus, node1plus
        return cost_matrix[cycle[node1-1], cycle[node2]] + cost_matrix[cycle[node2], cycle[node1]] + cost_matrix[cycle[node1], cycle[node2plus]] - cost_matrix[cycle[node1-1], cycle[node1]] - cost_matrix[cycle[node1], cycle[node2]] - cost_matrix[cycle[node2], cycle[node2plus]]

    return cost_matrix[cycle[node1-1], cycle[node2]] + cost_matrix[cycle[node2], cycle[node1plus]] + cost_matrix[cycle[node2-1], cycle[node1]] + cost_matrix[cycle[node1], cycle[node2plus]] - cost_matrix[cycle[node1-1], cycle[node1]] - cost_matrix[cycle[node1], cycle[node1plus]] - cost_matrix[cycle[node2-1], cycle[node2]] - cost_matrix[cycle[node2], cycle[node2plus]]

In [35]:
def intra_two_edges_exchange_delta(cycle, dist_matrix, node1, node2):
    if node1 == len(cycle)-1 or node2 == len(cycle)-1:
        return 99999
    #print(cost_matrix[cycle[node1], cycle[node2]], cost_matrix[cycle[node1+1], cycle[node2+1]], cost_matrix[cycle[node1], cycle[node1+1]], cost_matrix[cycle[node2], cycle[node2+1]])
    return dist_matrix[cycle[node1], cycle[node2]] + dist_matrix[cycle[node1+1], cycle[node2+1]] - dist_matrix[cycle[node1], cycle[node1+1]] - dist_matrix[cycle[node2], cycle[node2+1]]


In [36]:
jit()
def steepest_local_search(cost_matrix, dist_matrix, nodes_exchange = True, initial_greedy = False, starting_node = None):
    if initial_greedy:
        if starting_node:
            cycle = greedy_cycle(cost_matrix, starting_node)
        else:
            cycle = greedy_cycle(cost_matrix, random.randint(0, len(cost_matrix)-1))
    else:
        cycle = random_solution(cost_matrix)
    unused_nodes = set(list(range(0,len(cost_matrix))))
    for node in cycle:
        unused_nodes.remove(node)

    all_combinations = list(combinations(list(range(0, 100)), 2))

    while(True):
        best_delta, swap_node_a, swap_node_b, best_type = 0, None, None, None
        for used_cycle_id in range(len(cycle)):
            for unused_node in unused_nodes:
                delta = inter_exchange_delta(cycle, cost_matrix, used_cycle_id, unused_node)
                if delta < best_delta:
                    best_delta, swap_node_a, swap_node_b, best_type = delta, used_cycle_id, unused_node, 'inter_exchange'
        if nodes_exchange:
            for x, y in all_combinations:
                delta = intra_two_nodes_exchange_delta(cycle, cost_matrix, x, y)
                if delta < best_delta:
                    best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'intra_two_nodes_exchange'
        else:
            for x, y in all_combinations:
                delta = intra_two_edges_exchange_delta(cycle, dist_matrix, x, y)
                if delta < best_delta:
                    best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'intra_two_edges_exchange'

        if best_type is not None:
            if best_type == 'inter_exchange':
                unused_nodes.add(cycle[swap_node_a])
                unused_nodes.remove(swap_node_b)
                cycle[swap_node_a] = swap_node_b
            elif best_type == 'intra_two_nodes_exchange':
                cycle[swap_node_a], cycle[swap_node_b] = cycle[swap_node_b], cycle[swap_node_a]
            elif best_type == 'intra_two_edges_exchange':
                cycle[swap_node_a+1], cycle[swap_node_b] = cycle[swap_node_b], cycle[swap_node_a+1]
                cycle[swap_node_a+2:swap_node_b] = cycle[swap_node_a+2:swap_node_b][::-1]
        else:
            break
    return cycle

In [37]:
jit()
def greedy_local_search(cost_matrix, dist_matrix, nodes_exchange = True, initial_greedy = False, starting_node = None):
    if initial_greedy:
        if starting_node:
            cycle = greedy_cycle(cost_matrix, starting_node)
        else:
            cycle = greedy_cycle(cost_matrix, random.randint(0, len(cost_matrix)-1))
    else:
        cycle = random_solution(cost_matrix)
    unused_nodes = set(list(range(0,len(cost_matrix))))
    for node in cycle:
        unused_nodes.remove(node)

    #neighbourhood = [(x, y, 'inter') for x in range(len(cycle)) for y in unused_nodes]
    #neighbourhood += [(x, y, 'intra') for x, y in combinations(list(range(0, 100)), 2)]

    while(True):
        best_delta, swap_node_a, swap_node_b, best_type = 0, None, None, None
        neighbourhood = [(x, y, 'inter') for x in range(len(cycle)) for y in unused_nodes]
        neighbourhood += [(x, y, 'intra') for x, y in combinations(list(range(0, 100)), 2)]
        random.shuffle(neighbourhood)
        if nodes_exchange:
            for x, y, z in neighbourhood:
                if z == 'inter':
                    delta = inter_exchange_delta(cycle, cost_matrix, x, y)
                    if delta < best_delta:
                        best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'inter_exchange'
                        break
                elif z == 'intra':
                    delta = intra_two_nodes_exchange_delta(cycle, cost_matrix, x, y)
                    if delta < best_delta:
                        best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'intra_two_nodes_exchange'
                        break
        else:
            for x, y, z in neighbourhood:
                if z == 'inter':
                    delta = inter_exchange_delta(cycle, cost_matrix, x, y)
                    if delta < best_delta:
                        best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'inter_exchange'
                        break
                elif z == 'intra':
                    delta = intra_two_edges_exchange_delta(cycle, dist_matrix, x, y)
                    if delta < best_delta:
                        best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'intra_two_edges_exchange'
                        break

        if best_type is not None:
            if best_type == 'inter_exchange':
                unused_nodes.add(cycle[swap_node_a])
                unused_nodes.remove(swap_node_b)
                cycle[swap_node_a] = swap_node_b
                #neighbourhood = [(x, y) for x in range(len(cycle)) for y in unused_nodes]
            elif best_type == 'intra_two_nodes_exchange':
                cycle[swap_node_a], cycle[swap_node_b] = cycle[swap_node_b], cycle[swap_node_a]
            elif best_type == 'intra_two_edges_exchange':
                cycle[swap_node_a+1], cycle[swap_node_b] = cycle[swap_node_b], cycle[swap_node_a+1]
                cycle[swap_node_a+2:swap_node_b] = cycle[swap_node_a+2:swap_node_b][::-1]
        else:
            break
    return cycle

In [38]:
#steepest_local_search(cost_matrix_A, dist_matrix_A, inter = True, initial_greedy = False)
#greedy_local_search(cost_matrix_A, dist_matrix_A, inter = True, initial_greedy = False)

Tests

In [39]:
cyc = steepest_local_search(cost_matrix_C, dist_matrix_C, False, initial_greedy = False)
calculate_performance(cyc, cost_matrix_C)

C:\Users\hubra\AppData\Local\Temp\ipykernel_8412\3945026401.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "random_solution" failed type inference due to: No implementation of function Function(<bound method Random.shuffle of <random.Random object at 0x000001A54B4F5400>>) found for signature:
 
 >>> shuffle(list(int64)<iv=None>)
 
There are 2 candidate implementations:
      - Of which 2 did not match due to:
      Overload in function 'shuffle_impl': File: numba\cpython\randomimpl.py: Line 1230.
        With argument(s): '(list(int64)<iv=None>)':
       Rejected as the implementation raised a specific error:
         TypeError: The argument to shuffle() should be a buffer type
  raised from C:\Users\hubra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cpython\randomimpl.py:1206

During: resolving callee type: Function(<bound method Random.shuffl

52419

In [40]:
def present_results(min, max, average, time):
    print("{: >10} {: >10} {: >10} {: >10}".format('MIN', 'MAX', 'AVG', 'TIME'))
    print("{: >10} {: >10} {: >10} {: >10}".format(min, max, average, time))
    print()

In [41]:
# local_search(cost_matrix, inter = True, initial_greedy = False, starting_node = None)
def test_solution(local_search, cost_matrix, dist_matrix, inter, initial_greedy):
    start = time.time()
    costs = []
    for i in range(len(cost_matrix)):
        cycle = local_search(cost_matrix, dist_matrix, inter, initial_greedy, i)
        total_cost = calculate_performance(cycle, cost_matrix)
        costs.append(total_cost)
    costs = np.array(costs)
    return costs.min(), costs.max(), costs.mean(), time.time() - start

local search | neighborhood | starting solutions

In [42]:
for i in range(2,4):
    if i == 0:
        cost_matrix = cost_matrix_A
        dist_matrix = dist_matrix_A
        print('DATASET A')
    elif i == 1:
        cost_matrix = cost_matrix_B
        dist_matrix = dist_matrix_B
        print('DATASET B')
    elif i == 2:
        cost_matrix = cost_matrix_C
        dist_matrix = dist_matrix_C
        print('DATASET C')
    elif i == 3:
        cost_matrix = cost_matrix_D
        dist_matrix = dist_matrix_D
        print('DATASET D')

    print('STEEPEST | NODES | GREEDY')
    present_results(*test_solution(steepest_local_search, cost_matrix, dist_matrix, True, True))
    print('STEEPEST | NODES | RANDOM')
    present_results(*test_solution(steepest_local_search, cost_matrix, dist_matrix, True, False))
    print('STEEPEST | EDGES | GREEDY')
    present_results(*test_solution(steepest_local_search, cost_matrix, dist_matrix, False, True))
    print('STEEPEST | EDGES | RANDOM')
    present_results(*test_solution(steepest_local_search, cost_matrix, dist_matrix, False, False))
    print('GREEDY | NODES | GREEDY')
    present_results(*test_solution(greedy_local_search, cost_matrix, dist_matrix, True, True))
    print('GREEDY | NODES | RANDOM')
    present_results(*test_solution(greedy_local_search, cost_matrix, dist_matrix, True, False))
    print('GREEDY | EDGES | GREEDY')
    present_results(*test_solution(greedy_local_search, cost_matrix, dist_matrix, False, True))
    print('GREEDY | EDGES | RANDOM')
    present_results(*test_solution(greedy_local_search, cost_matrix, dist_matrix, False, False))
    print('===================================')

DATASET C
STEEPEST | NODES | GREEDY
       MIN        MAX        AVG       TIME
     53080      58520   55471.88 39.320133686065674

STEEPEST | NODES | RANDOM
       MIN        MAX        AVG       TIME
     58375      75319  65464.805 744.2845809459686

STEEPEST | EDGES | GREEDY
       MIN        MAX        AVG       TIME
     52889      58513  55201.485 37.82859992980957

STEEPEST | EDGES | RANDOM
       MIN        MAX        AVG       TIME
     49056      54507   51678.23 471.44153475761414

GREEDY | NODES | GREEDY
       MIN        MAX        AVG       TIME
     53080      58520   55464.68 28.637354850769043

GREEDY | NODES | RANDOM
       MIN        MAX        AVG       TIME
     58581      69919    63930.4 662.9003674983978

GREEDY | EDGES | GREEDY
       MIN        MAX        AVG       TIME
     53079      58513  55215.395 32.189746618270874

GREEDY | EDGES | RANDOM
       MIN        MAX        AVG       TIME
     49471      55058  51770.755 907.2033560276031

DATASET D
STEEPEST